In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np

from adaptable import AdaptableNet

In [2]:
# Set seed properly
np.random.seed(1)
torch.manual_seed(1)

# Create the AdaptableNet
test_net = AdaptableNet(12, 5, hidden_size=[7, 10])
# Create some fake data
test_data = torch.Tensor(np.random.random((4, 12)))
# Generate some fake output
test_output = test_net(test_data)

# Increase the size of the first hidden layer of the net
test_net.increase_hidden_size(0)

print("Difference After Increase 1:", torch.norm(test_output - test_net(test_data)))

# Immediately decrease the size of the first hidden layer of the net 
# Note: this first decrease will be a trivial operation since we are just undoing the increase from above
test_net.decrease_hidden_size(0)

print("Difference After Decrease 1:", torch.norm(test_output - test_net(test_data)))

# Decrease the size of the first hidden layer of the net AGAIN. 
# This will be a nontrivial decrease, as it has to get rid of some nonzero weights
test_net.decrease_hidden_size(0)

print("Difference After Decrease 2:", torch.norm(test_output - test_net(test_data)))

Difference After Increase 1: tensor(0., grad_fn=<CopyBackwards>)
Difference After Decrease 1: tensor(0., grad_fn=<CopyBackwards>)
Difference After Decrease 2: tensor(0.0399, grad_fn=<CopyBackwards>)


Experiment 1: Seeing what method of decreasing the hidden size is in general best. May need to come back to try new methods of doing this since the current methods are very heuristic.

In [ ]:
# Will store our "losses" for each mode
losses = np.zeros((4, 1000))

# For different seeds
for i in range(1000):
    # For each mode
    for mode in range(4):
        # Set the seeds
        np.random.seed(i)
        torch.manual_seed(i)
        # Initialize Net
        test_net = AdaptableNet(12, 5, hidden_size=[10,])
        # Generate data
        test_data = torch.Tensor(np.random.random((4, 12)))
        # Generate output
        test_output = test_net(test_data)
        # Decrease the size
        test_net.decrease_hidden_size(0, mode=mode)
        # Compute the norm of the difference in the output
        losses[mode, i] = torch.norm(test_output - test_net(test_data))
        
means = np.mean(losses, axis=1)
print("Method 0 avg. loss:", means[0])
print("Method 1 avg. loss:", means[1])
print("Method 2 avg. loss:", means[2])
